In [2]:
import sys
import random
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    # lines_series = [line[1] for line in line_atom]
    # lines_series = np.array(lines_series).astype(int)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position



def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    # all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):

            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            # all_distance[i][j] = _distance

            if (_distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])

    return np.array(pair)

def cal_domak(lines_index, pair):

    max_split_value = 0
    _counter = 0

    for current in lines_index:
        int_A = np.sum((pair[:,0]<current) & (pair[:,1]<current))
        int_B = np.sum((pair[:,0]>current) & (pair[:,1]>current))
        ext_AB = np.sum((pair[:,0]<=current) & (pair[:,1]>=current))

        if ext_AB != 0:
            _score = (int_A/ext_AB) * (int_B/ext_AB)

            if max_split_value < _score:
                max_split_value = _score
                split_index = current
                index = _counter
        _counter+=1
    
    return max_split_value, split_index, index

# pdb_path = sys.argv[1]
pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
pair = cal_distance(lines_index, lines_position)
max_split_value, split_index, index = cal_domak(lines_index, pair)

In [64]:
import sys
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position


def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    # all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):

            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            # all_distance[i][j] = _distance

            if (_distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])

    return np.array(pair)



def split_group(lines_index, lines_position):

    def get_swap(group_1, group_2):
        max_count = 0
        for i in group_1.keys():
            _simple_count = 0
            for j in group_2.keys():
                _distance = np.sqrt(np.sum( (group_1[i]-group_2[j])**2 ))
                if (_distance<=threshold):
                    _simple_count += 1

            if max_count < _simple_count:
                max_count = _simple_count
                _swap = i
        return max_count, _swap


    dic_position = dict(zip(lines_index, lines_position))
    _split = int(len(lines_position)/2)
    threshold = 10

    sorted = []

    group_U = dict(list(dic_position.items())[:_split] )
    group_V = dict(list(dic_position.items())[_split:] )

    for _ in range(int(len(lines_position)/2)):
        print('\r' + 'Processing {:.2f}%'.format(_/len(lines_position)*100), end='', flush=True)
        max_count1, _swap1 = get_swap(group_U,group_V)
        max_count2, _swap2 = get_swap(group_V,group_U)
        
        if max_count1 > max_count2:
            group_U.pop(_swap1)
            group_V[_swap1] = dic_position[_swap1]

            
        else:
            group_V.pop(_swap2)
            group_U[_swap2] = dic_position[_swap2]

        sorted.append(_swap1)
        sorted.append(_swap2)

    
    best_combine1 = list(set(list(group_U.keys())))
    best_combine2 = list(set(list(group_V.keys())))
            
    print()
    print("Group 1")
    print( best_combine1 )
    print("Group 2")
    print( best_combine2 )

    return best_combine1, best_combine2

In [65]:
pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
pair = cal_distance(lines_index, lines_position)
best_combine1, best_combine2 = split_group(lines_index, lines_position)

Processing 49.49%
Group 1
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 118, 121, 122, 124, 125, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156]
Group 2
[102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 123, 126, 127, 128, 129, 130, 131, 132, 133, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 21